In [ ]:
from openpyxl import load_workbook
import random
from docx import Document
from num2words import num2words

def informacoes_iniciais(escola):
  meses= ("","Janeiro","Fevereiro","Março","Abril","Maio","Junho","Julho","Agosto","Setembro","Outubro","Novembro","Dezembro")
  cnpj_buscar = str(input("Digite o CNPJ da escola que deseja bucar(sem . ou /, ex:123456789) : "))
  nome_escola,cnpj_escola,cep_escola,cidade_escola,endereco_escola,diretor_escola = buscar_escola(escola,cnpj_buscar)
  print(f"\n este cnpj corresponde a escola {nome_escola}\n")
  dia = str(input("Digite a data dos orçamentos (dia): "))
  mes = int(input("Digite a data dos orçamentos (mês): "))
  if mes<1 or mes>12:
    print("o numero do mês está errado")
    exit(0)
  mes_digitado = meses[mes]
  ano = str(input("Digite a data dos orçamentos (ano): "))
  NF = str(input("Digite o número da nota fiscal correspondente : "))
  return nome_escola,cnpj_escola,cep_escola,cidade_escola,endereco_escola,dia,mes,mes_digitado,ano,NF,diretor_escola,meses

def opcoes():
  dia_emitida = mes_emitida = ano_emitida = None
  opcao_consolidacao = str(input("Você quer consolidação? (S/N) "))
  if opcao_consolidacao.lower() == "s":
    dia_consolidacao = int(input("Digite a data da consolidação (dia): "))
    mes_consolidacao = int(input("Digite a data da consolidação (mês): "))
    ano_consolidacao = str(input("Digite a data da consolidação (ano): "))


  opcao_recibo = str(input("Você quer recibo? (S/N) "))
  if opcao_recibo.lower() == "s":
    dia_recibo = int(input("Digite a data do recibo (dia): "))
    mes_recibo = int(input("Digite a data do recibo (mês): "))
    ano_recibo = str(input("Digite a data do recibo (ano): "))
    nota = str(input("a data que a nota foi emitida é a mesma do recibo? (S/N)"))
    if nota.lower() == "n":
      dia_emitida = int(input("Digite a data da emissão (dia): "))
      mes_emitida = int(input("Digite a data da emissão (mês): "))
      ano_emitida = int(input("Digite a data da emissão (ano): "))
  meio_pagamento = str(input("Pago por meio de:  "))
  return opcao_consolidacao,dia_consolidacao,mes_consolidacao,ano_consolidacao,opcao_recibo,dia_recibo,mes_recibo,ano_recibo,dia_emitida,mes_emitida,ano_emitida,meio_pagamento

def buscar_escola(escola,buscador):
  for row in range(2,escola.max_row + 1):
    celula_buscar = escola.cell(row=row,column = 1).value
    if celula_buscar == buscador:
      nome_escola = escola.cell(row=row,column = 2).value
      cnpj_escola = escola.cell(row=row,column = 3).value
      cep_escola = escola.cell(row=row,column = 4).value
      cidade_escola = escola.cell(row=row,column = 5).value
      endereco_escola = escola.cell(row=row,column = 6).value
      diretor_escola = escola.cell(row=row,column = 7).value
      return nome_escola,cnpj_escola,cep_escola,cidade_escola,endereco_escola,diretor_escola
  print("Escola não encontrada")
  exit(0)

def aplicar_informacoes(pagina_escolas,nce,paper,grafite):
  nome_escola,cnpj_escola,cep_escola,cidade_escola,endereco_escola,dia,mes,mes_digitado,ano,NF,diretor_escola,meses =informacoes_iniciais(pagina_escolas)
  #nce
  pagina_nce.cell(row=6,column=1).value = f"NOSSA SENHORA DA GLÓRIA/SE {dia} DE {mes_digitado.upper()} DE {ano}"
  pagina_nce.cell(row=8,column=1).value = nome_escola
  pagina_nce.cell(row=9,column=1).value = endereco_escola
  pagina_nce.cell(row=10,column=1).value = f"CEP {cep_escola} - {cidade_escola}"
  pagina_nce.cell(row=11,column=1).value = f"CNPJ: {cnpj_escola}"
  #paper&co
  pagina_paper.cell(row=9,column=1).value = f"NOSSA SENHORA DAS DORES/SE, {dia} DE {mes_digitado.upper()} DE {ano}"
  pagina_paper.cell(row=11,column=1).value = f"{nome_escola}      /      CNPJ- {cnpj_escola}"
  pagina_paper.cell(row=12,column=1).value = endereco_escola
  pagina_paper.cell(row=13,column=1).value = f"CEP {cep_escola}   -   {cidade_escola}"
  #grafite
  pagina_grafite.cell(row=7,column=2).value = nome_escola.title()
  pagina_grafite.cell(row=7,column=5).value = cnpj_escola
  pagina_grafite.cell(row=8,column=2).value = endereco_escola.title()
  pagina_grafite.cell(row=9,column=2).value = cidade_escola.title()
  pagina_grafite.cell(row=10,column=2).value = cep_escola
  pagina_grafite.cell(row=10,column=5).value = f"{dia} DE {mes_digitado.upper()} DE {ano}"

  return dia,mes,ano,NF,diretor_escola,cidade_escola,meses,nome_escola,cnpj_escola

def numero_por_extenso(num):
  try:
    return num2words(num, lang='pt_BR')
  except NotImplementedError:
    return "Número muito grande ou formato não suportado pela biblioteca."
  except Exception as e:
    return f"Ocorreu um erro: {e}"

def fazer_recibo(arquivo_recibo,total_nce,nome_escola,NF,dia_emitida,mes_emitida,ano_emitida,dia_recibo,mes_recibo,ano_recibo,meio_pagamento,meses):
  for paragrafo in arquivo_recibo.paragraphs:
    if "<VALOR>" in paragrafo.text:
      paragrafo.text = paragrafo.text.replace("<VALOR>",round(total_nce,2))
    if "<EXTENSO>" in paragrafo.text:
      paragrafo.text = paragrafo.text.replace("<EXTENSO>",numero_por_extenso(float(total_nce)))
    if "<NOME>" in paragrafo.text:
      paragrafo.text = paragrafo.text.replace("<NOME>",nome_escola)
    if "<NF>" in paragrafo.text:
      paragrafo.text = paragrafo.text.replace("<NF>",NF)
    if "<DATA_EMISSÃO>" in paragrafo.text:
      if dia_emitida and mes_emitida and ano_emitida:
        paragrafo.text = paragrafo.text.replace("<DATA_EMISSÃO>",f"{dia_emitida}/{mes_emitida}/{ano_emitida}")
      else:
        paragrafo.text = paragrafo.text.replace("<DATA_EMISSÃO>",f"{dia_recibo}/{mes_recibo}/{ano_recibo}")
    if "<MEIO>" in paragrafo.text:
      paragrafo.text = paragrafo.text.replace("<MEIO>",meio_pagamento)
    if "<DATA>" in paragrafo.text:
      paragrafo.text = paragrafo.text.replace("<DATA>",f"{dia_recibo}/{mes_recibo}/{ano_recibo}")
    if "<DIA>" in paragrafo.text:
      paragrafo.text = paragrafo.text.replace("<DIA>",dia_recibo)
    if "<MÊS>" in paragrafo.text:
      paragrafo.text = paragrafo.text.replace("<MÊS>",meses[mes_recibo])
    if "<ANO>" in paragrafo.text:
      paragrafo.text = paragrafo.text.replace("<ANO>",ano_recibo)

def fazer_consolidacao(arquivo_consolidacao,item_numero,produto,un,qt,unit_nce,unit_paper,unit_grafite,diretor_escola,
                       nome_escola,dia_consolidacao,mes_consolidacao,ano_consolidacao,cidade_escola,cnpj_escola,total_nce,total_paper,total_grafite,meses):
    for paragrafo in arquivo_consolidacao.paragraphs:
      if f"<VALOR{item_numero}>" in paragrafo.text:
        paragrafo.text = paragrafo.text.replace(f"<VALOR{item_numero}",str(produto))
      if f"<UNIT{item_numero}>" in paragrafo.text:
        paragrafo.text = paragrafo.text.replace(f"<UNIT{item_numero}",str(un))
      if f"<QUANT{item_numero}>" in paragrafo.text:
        paragrafo.text = paragrafo.text.replace(f"<QUANT{item_numero}",str(qt))
      if f"<VALOR_A{item_numero}>" in paragrafo.text:
        paragrafo.text = paragrafo.text.replace(f"<VALOR_A{item_numero}",str(unit_nce))
      if f"<VALOR_B{item_numero}>" in paragrafo.text:
        paragrafo.text = paragrafo.text.replace(f"<VALOR_B{item_numero}",str(unit_paper))
      if f"<VALOR_C{item_numero}>" in paragrafo.text:
        paragrafo.text = paragrafo.text.replace(f"<VALOR_C{item_numero}",str(unit_grafite))
      if "<DIRETOR>" in paragrafo.text:
        paragrafo.text = paragrafo.text.replace("DIRETOR",diretor_escola)
      if "<CIDADE>" in paragrafo.text:
        paragrafo.text = paragrafo.text.replace("CIDADE",cidade_escola)
      if "<DATA>" in paragrafo.text:
        paragrafo.text = paragrafo.text.replace("DATA",f"{dia_consolidacao} de {meses[mes_consolidacao]} de {ano_consolidacao}")
      if "<TOTAL_A>" in paragrafo.text:
        paragrafo.text = paragrafo.text.replace("TOTAL_A",round(total_nce,2))
      if "<TOTAL_B>" in paragrafo.text:
        paragrafo.text = paragrafo.text.replace("TOTAL_B",total_paper)
      if "<TOTAL_C>" in paragrafo.text:
        paragrafo.text = paragrafo.text.replace("TOTAL_C",total_grafite)
      if "<NOME>" in paragrafo.text:
        paragrafo.text = paragrafo.text.replace("NOME",nome_escola)
      if "<CNPJ>" in paragrafo.text:
        paragrafo.text = paragrafo.text.replace("CNPJ",cnpj_escola)

arquivo_nce = load_workbook("MODELO NCE.xlsx")
pagina_nce = arquivo_nce["Matriz"]

arquivo_paper = load_workbook("MODELO PAPER.xlsx")
pagina_paper = arquivo_paper.active

arquivo_grafite = load_workbook("MODELO GRAFITE.xlsx")
pagina_grafite = arquivo_grafite.active

arquivo_consolidacao = Document("MODELO CONSOLIDACAO.docx")

arquivo_recibo = Document("MODELO RECIBO.docx")

arquivo_controle = load_workbook("MODELO CONTROLE.xlsx")
pagina_controle = arquivo_controle.active

arquivo_escolas = load_workbook("escolas.xlsx")
pagina_escolas = arquivo_escolas.active
dia,mes,ano,NF,diretor_escola,cidade_escola,meses,nome_escola,cnpj_escola = aplicar_informacoes(pagina_escolas,pagina_nce,pagina_paper,pagina_grafite)
opcao_consolidacao,dia_consolidacao,mes_consolidacao,ano_consolidacao,opcao_recibo,dia_recibo,mes_recibo,ano_recibo,dia_emitida,mes_emitida,ano_emitida,meio_pagamento = opcoes()

arquivo_doador = load_workbook("NELSON.xlsx")
pagina_doador = arquivo_doador["Table 10"]

linha_ini_nce = 15
linha_ini_paper = 17
linha_ini_grafite = 12
linha_ini_controle = 3
linha_doadora = 3

precos_randomizados_por_valor = {}

total_nce = 0
total_paper = 0
total_grafite = 0
item_numero = 1
while pagina_doador.cell(row=linha_doadora,column=2).value is not None:
  produto = pagina_doador.cell(row=linha_doadora,column=2).value
  un = pagina_doador.cell(row=linha_doadora,column=6).value
  qt = int(pagina_doador.cell(row=linha_doadora,column=7).value)
  unit_nce = float(pagina_doador.cell(row=linha_doadora,column=8).value)

  if unit_nce in precos_randomizados_por_valor:
    unit_paper,unit_grafite = precos_randomizados_por_valor[unit_nce]
  else:
   unit_paper = unit_nce * (1+ random.uniform(0.15,0.25))
   unit_grafite = unit_nce * (1+ random.uniform(0.15,0.25))
   while abs(unit_paper - unit_grafite)< 0.10:
     unit_grafite = unit_nce * (1+ random.uniform(0.15,0.25))
   precos_randomizados_por_valor[unit_nce] = unit_paper,unit_grafite


  pagina_nce.cell(row=linha_ini_nce,column=2).value = produto
  pagina_nce.cell(row=linha_ini_nce,column=3).value = un
  pagina_nce.cell(row=linha_ini_nce,column=4).value = qt
  pagina_nce.cell(row=linha_ini_nce,column=5).value = unit_nce
  total_nce = total_nce +(unit_nce*qt)


  pagina_paper.cell(row=linha_ini_paper,column=2).value = produto
  pagina_paper.cell(row=linha_ini_paper,column=3).value = un
  pagina_paper.cell(row=linha_ini_paper,column=4).value = qt
  pagina_paper.cell(row=linha_ini_paper,column=5).value = unit_paper
  total_paper = total_paper +(unit_paper*qt)

  pagina_grafite.cell(row=linha_ini_grafite,column=2).value = produto
  pagina_grafite.cell(row=linha_ini_grafite,column=3).value = un
  pagina_grafite.cell(row=linha_ini_grafite,column=4).value = qt
  pagina_grafite.cell(row=linha_ini_grafite,column=5).value = unit_grafite
  total_grafite = total_grafite +(unit_grafite*qt)

  pagina_controle.cell(row=linha_ini_controle,column=2).value = produto
  pagina_controle.cell(row=linha_ini_controle,column=3).value = un
  pagina_controle.cell(row=linha_ini_controle,column=4).value = qt
  pagina_controle.cell(row=linha_ini_controle,column=5).value = unit_nce
  pagina_controle.cell(row=linha_ini_controle,column=8).value = unit_paper
  pagina_controle.cell(row=linha_ini_controle,column=11).value = unit_grafite

  #CONSOLIDAÇÃO
  if opcao_consolidacao.lower() == "s":
     fazer_consolidacao(arquivo_consolidacao,item_numero,produto,un,qt,unit_nce,unit_paper,unit_grafite,diretor_escola,
                        nome_escola,dia_consolidacao,mes_consolidacao,ano_consolidacao,cidade_escola,cnpj_escola,total_nce,total_paper,total_grafite,meses)

  item_numero += 1

if opcao_recibo.lower() == "s":
  fazer_recibo(arquivo_recibo,total_nce,nome_escola,NF,dia_emitida,mes_emitida,ano_emitida,dia_recibo,mes_recibo,ano_recibo,meio_pagamento,meses)

  linha_doadora +=1
  linha_ini_grafite +=1
  linha_ini_paper +=1
  linha_ini_nce +=1
  linha_ini_controle +=1

arquivo_nce.save(f"ORÇAMENTO NF{NF} {ano}-{mes}-{dia} NCE.xlsx")
arquivo_paper.save(f"ORÇAMENTO NF{NF} {ano}-{mes}-{dia} PAPER&CO.xlsx")
arquivo_grafite.save(f"ORÇAMENTO NF{NF} {ano}-{mes}-{dia} GRAFITE.xlsx")
arquivo_controle.save(f"MODELO DOC NF{NF}.xlsx")
arquivo_consolidacao.save(f"ORÇAMENTO NF{NF} {ano_consolidacao}-{mes_consolidacao}-{dia_consolidacao} CONSOLIDAÇÃO DE PESQ DE PREÇO")
arquivo_recibo.save(f"ORÇAMENTO NF{NF} {ano_recibo}-{mes_recibo}-{dia_recibo} RECIBO {round(total_nce,2)} NCE")


Digite o CNPJ da escola que deseja bucar(sem . ou /, ex:123456789) : 1960308000141

 este cnpj corresponde a escola CONSELHO ESCOLAR COL EST NELSON RESENDE ALBUQUERQUE

Digite a data dos orçamentos (dia): 09
Digite a data dos orçamentos (mês): 07
Digite a data dos orçamentos (ano): 2025
Digite o número da nota fiscal correspondente : 10123
